In [1]:
#get daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import os
df=pd.DataFrame()
for f in os.listdir(r'./DataSource/baiduMap')[-10:]:
    print(f)
    d=pd.read_csv(r'./DataSource/baiduMap/'+f, on_bad_lines='skip')
    df=pd.concat([df,d])

# df=df.rename(columns={'抓取日期':'test','抓取时段':'抓取日期','路况语义化描述':'抓取时段'})
df['抓取日期']=pd.to_datetime(df['抓取日期']).dt.date
df=df.drop_duplicates(subset=['城市','商场名称','抓取日期','抓取时段'])

df.tail(2)

baiduTrafficV2_2023-01-08.csv
baiduTrafficV2_2023-01-09.csv
baiduTrafficV2_2023-01-10.csv
baiduTrafficV2_2023-01-11.csv
baiduTrafficV2_2023-01-12.csv
baiduTrafficV2_2023-01-13.csv
baiduTrafficV2_2023-01-14.csv
baiduTrafficV2_2023-01-15.csv
baiduTrafficV2_2023-01-16.csv
baiduTrafficV2_2023-01-17.csv


,编号,省份,城市,城市线,商场名称,经度,纬度,抓取日期,抓取时段,路况语义化描述,路况状态,路径整体评价,附近道路数量,样本道路数量,平均通行速度（KM/H),radius
856,270,新疆,克拉玛依,三线及以下,汇嘉时代（准噶尔路店）,84.857470,45.602009,2023-01-17,10,该区域整体畅通。,1.0,畅通,5,0,NaN,200
857,258,云南,保山,三线及以下,保山五洲国际广场,99.178607,25.138764,2023-01-17,10,该区域整体畅通。,1.0,畅通,3,0,NaN,200


In [2]:
old=pd.read_hdf(r'Processed_Data/mall.h5',key='0105')
df=pd.concat([old,df])
df=df.drop_duplicates()
df.to_hdf(r'Processed_Data/mall.h5',key='0105')

In [8]:
#copy to wkbk
d1=pd.read_hdf('Processed_Data/Mallcity.h5','raw')
d2=pd.pivot_table(df[df['抓取日期'].astype('str')>='2023-01-13'],index=['城市线','城市','商场名称'],columns=['抓取日期','抓取时段'],values='路况状态',aggfunc='max').reset_index().fillna(1)
d3=d1.merge(d2,how='left',on=['城市线', '城市', '商场名称'])
d3.drop(columns=['城市线', '城市', '商场名称','序号']).to_clipboard(index=False)

In [7]:
#charts
import numpy as np

d=df.copy()
c1=d['城市'].isin(d1['城市'].drop_duplicates())
c2=d['省份'].isin(d1['城市'].drop_duplicates())

d=d[c1|c2]

c3=d['抓取时段'].isin([10,12,14])
d=d[~c3]

d_N=d.groupby(by=['抓取日期','抓取时段'])['编号'].count().reset_index().rename(columns={'编号':'N'})

d=d.groupby(by=['抓取日期','抓取时段','路径整体评价'])['编号'].count().reset_index()

d=d[d['路径整体评价']=='畅通']
d=d.merge(d_N,on=['抓取日期','抓取时段'],how='left')
d['拥堵率']=1-(d['编号']/d['N'])

d=d.sort_values(by=['抓取日期','抓取时段'],ascending=True)
d['拥堵率_last7d_mean']=d.groupby(by='抓取时段')['拥堵率'].transform(lambda x: x.rolling(7, 1).mean())
show=pd.pivot_table(d,index=['抓取日期'],columns='抓取时段',values='拥堵率_last7d_mean',aggfunc='mean')
show['sum']=np.nan
cols=[show.columns[-1]]+list(show.columns[:-1])
show=show[cols]
show.drop(columns=16).to_clipboard(header=False)

In [5]:
d

,抓取日期,抓取时段,路径整体评价,编号,N,拥堵率,拥堵率_last7d_mean
0,2022-12-01,20,畅通,292,296,0.013514,0.013514
1,2022-12-02,16,畅通,288,296,0.027027,0.027027
2,2022-12-02,20,畅通,287,296,0.030405,0.021959
3,2022-12-03,16,畅通,285,296,0.037162,0.032095
4,2022-12-03,20,畅通,292,296,0.013514,0.019144
...,...,...,...,...,...,...,...
86,2023-01-14,20,畅通,285,296,0.037162,0.049228
87,2023-01-15,16,畅通,248,296,0.162162,0.111969
88,2023-01-15,20,畅通,280,296,0.054054,0.050193
89,2023-01-16,16,畅通,250,296,0.155405,0.123069
